# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [50]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [51]:
from sklearn.model_selection import train_test_split


In [60]:
# Separate features (X) and target (Y)
X = adult_dt.drop(columns=['income'])  # X contains all columns except 'income'
Y = adult_dt['income']                 # Y contains only the 'income' column

# Split the data into training and testing sets (70-30 split)
X0_train, X0_test, Y0_train, Y0_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

1) The random_state parameter in scikit-learn’s train_test_split function sets the seed for the random number generator used to shuffle and split the data into training and test sets. When random_state is specified (e.g., random_state=42), the random process that shuffles and splits the data will be consistent every time the code is run. This parameter can be set to any integer.

Here’s a breakdown of what happens when random_state is used:

a-  If random_state is set to a specific integer, like random_state=42, the function uses this value as a "seed" for the random number generator, which controls the shuffling.
b- If random_state is not set (or set to None), the function will use the system’s time or another internal seed, and the split will vary each time the code is run.
The use of random_state ensures that the split of the dataset into training and test sets is deterministic. In other words, with a specific random_state, the same data points will consistently appear in either the training or the test set, enabling consistent results.

2) The concept of random_state is closely related to reproducibility. Reproducibility in data science and machine learning means that other researchers or stakeholders can run your code and obtain the same results as you did, assuming they use the same data and environment.

When training machine learning models, achieving reproducibility is crucial because it:

a- Allows verification of results: When someone else runs the same experiment, they can obtain identical training and test splits, ensuring that they observe the same performance metrics (accuracy, precision, etc.) that were initially reported.

b- Facilitates debugging and development: By fixing the randomness with random_state, developers can consistently reproduce the data splits and check for issues in specific parts of the dataset or model code.

c- Supports comparisons across models: Fixing the random state allows for a fair comparison between different model configurations or algorithms since they are trained and tested on identical data subsets.

d- If random_state is not set, each run might yield slightly different results due to the randomness in data splitting. This can make it challenging to identify issues or confidently report metrics, as they may fluctuate due to different splits.

In summary, random_state is useful because it ensures reproducibility, making results more reliable and comparisons between models or experiments more meaningful

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Define numerical and categorical feature lists
numerical_features = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]
categorical_features = [
    'workclass', 'education', 'marital-status', 'occupation', 
    'relationship', 'race', 'sex', 'native-country'
]

# Define the transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),  # KNN-based imputation
    ('scaler', RobustScaler())                                   # Robust scaling for outliers
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),         # Impute with mode (most frequent value)
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  # One-hot encoding
])

# Define the ColumnTransformer
transformer = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Drop any other columns not specified in transformers
)

# Create a pipeline with the transformer and a Decision Tree classifier
pipe = Pipeline([
    ( '-', transformer)
])
pipe

Pipeline(steps=[('-',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])]))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [55]:

# Define the pipeline
pipeline = Pipeline([
    ('preprocessing', transformer),         # Use the ColumnTransformer from previous section
    ('classifier', RandomForestClassifier()) # Add RandomForestClassifier as the classifier step
])

pipeline


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [62]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, log_loss
from sklearn.metrics import balanced_accuracy_score


In [69]:
# Split the data
X_rest, X_test, Y_rest, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train, X_validate, Y_train,  Y_validate = train_test_split(X_rest, Y_rest, test_size=0.3, random_state=42)

In [ ]:
Y_pred_train = pipe1.predict(X0_train)
Y_pred_test = pipe1.predict(X0_test)
Y_poba_train = pipe1.predict_proba(X0_train)
Y_proba_test = pipe1.predict_proba(X0_test)

In [65]:

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss', 
    'roc_auc': 'roc_auc', 
    'accuracy': 'accuracy',
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation with the pipeline
cv_results = cross_validate(
    pipeline, 
    X_rest, 
    Y_rest, 
    cv=5,              # 5-fold cross-validation
    scoring=scoring,   # Use the defined scoring metrics
    return_train_score=True  # To get training scores as well
)



c:\Users\user\.conda\envs\scale2prod\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\user\.conda\envs\scale2prod\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [66]:
# Define a separator for clarity
separator = "-" * 40

# Print the results with formatting
print("Model Cross-Validation Results\n" + separator)

# Negative Log Loss
print(f"Training Scores (Negative Log Loss): {cv_results['train_neg_log_loss']}")
print(f"Validation Scores (Negative Log Loss): {cv_results['test_neg_log_loss']}")
print(separator)

# ROC AUC
print(f"Training Scores (ROC AUC): {cv_results['train_roc_auc']}")
print(f"Validation Scores (ROC AUC): {cv_results['test_roc_auc']}")
print(separator)

# Accuracy
print(f"Training Scores (Accuracy): {cv_results['train_accuracy']}")
print(f"Validation Scores (Accuracy): {cv_results['test_accuracy']}")
print(separator)

# Balanced Accuracy
print(f"Training Scores (Balanced Accuracy): {cv_results['train_balanced_accuracy']}")
print(f"Validation Scores (Balanced Accuracy): {cv_results['test_balanced_accuracy']}")
print(separator)

Model Cross-Validation Results
----------------------------------------
Training Scores (Negative Log Loss): [-0.08130466 -0.08095565 -0.08103879 -0.08209355 -0.08167488]
Validation Scores (Negative Log Loss): [-0.34138496 -0.35603223 -0.37530166 -0.34382142 -0.41022861]
----------------------------------------
Training Scores (ROC AUC): [0.99999998 1.         1.         0.99999999 1.        ]
Validation Scores (ROC AUC): [0.90590932 0.90174503 0.90246543 0.90715635 0.90065888]
----------------------------------------
Training Scores (Accuracy): [0.99989031 0.99994515 0.99994516 0.99972579 1.        ]
Validation Scores (Accuracy): [0.85128318 0.84645756 0.85256692 0.85783238 0.8569548 ]
----------------------------------------
Training Scores (Balanced Accuracy): [0.99977381 0.9998869  0.99988693 0.99943464 1.        ]
Validation Scores (Balanced Accuracy): [0.77584054 0.76589479 0.77284937 0.7812476  0.77728384]
----------------------------------------


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [67]:
# Convert cv_results to a pandas DataFrame for display and sorting
cv_results_df = pd.DataFrame(cv_results)

# Sort by 'test_neg_log_loss' in ascending order to prioritize the folds with the best (least negative) log loss
cv_results_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted DataFrame
cv_results_sorted

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
4,7.885813,0.248284,-0.410229,-0.081675,0.900659,1.0,0.856955,1.000000,0.777284,1.000000
2,7.957775,0.189699,-0.375302,-0.081039,0.902465,1.0,0.852567,0.999945,0.772849,0.999887
1,7.625127,0.157768,-0.356032,-0.080956,0.901745,1.0,0.846458,0.999945,0.765895,0.999887
3,7.478170,0.162132,-0.343821,-0.082094,0.907156,1.0,0.857832,0.999726,0.781248,0.999435
0,8.440123,0.204418,-0.341385,-0.081305,0.905909,1.0,0.851283,0.999890,0.775841,0.999774


Calculate the mean of each metric. 

In [68]:
# Calculate the mean of each metric
mean_metrics = cv_results_df.mean()

# Display the mean metrics
mean_metrics

fit_time                   7.877402
score_time                 0.192460
test_neg_log_loss         -0.365354
train_neg_log_loss        -0.081414
test_roc_auc               0.903587
train_roc_auc              1.000000
test_accuracy              0.853019
train_accuracy             0.999901
test_balanced_accuracy     0.774623
train_balanced_accuracy    0.999796
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [73]:
# Generate predictions and prediction probabilities on the test data
Y_pred = pipeline.predict(X_test)
Y_pred_proba = pipeline.predict_proba(X_test)

# Calculate each performance metric
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display the test metrics as a dictionary
test_metrics

{'neg_log_loss': -0.38664823137003407,
 'roc_auc': 0.8987407301776664,
 'accuracy': 0.8548469648889344,
 'balanced_accuracy': 0.7758560003060715}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

This line of code is convenient for several reasons, especially when performing binary classification tasks in machine learning:

1) Binary Classification Requirement: The original income variable in the dataset contains string values ('>50K' and '<50K'). In most machine learning algorithms, it’s more efficient and appropriate to have the target variable in numeric form, where 1 represents the positive class (>50K) and 0 represents the negative class (<=50K). Recoding it in this way makes it easier to apply algorithms that require numeric targets for classification tasks.

2) Scikit-learn Compatibility: Scikit-learn’s classifiers expect target variables to be numeric, especially for binary classification tasks. By recoding the target as 1 and 0, we make it compatible with models that need numeric target labels (such as logistic regression, decision trees, random forests, etc.), as well as with performance metrics that expect numeric labels.

3) Simplified Preprocessing: The recoding is done in a single line of code using the assign() method with a lambda function. This reduces the need for additional steps in data preprocessing, making the code more concise and easier to read.

4) Consistent Format: By stripping the string and converting it into binary form, this ensures that the target variable is consistently formatted for downstream tasks like training, validation, and evaluation. This reduces the chances of errors or inconsistencies during the modeling process

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.